In [1]:
import pandas as pd
import numpy as np
import math
import json
% matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

### Define所有函数：

In [2]:
# offer id和offer_id这两列重复了，需要把他们合并：如果offer_id为空，则把offer id的值填到offer_id里
def combine_offer_id(a, b):
    if pd.isna(b):
        return a
    else:
        return b
    
    
# 创建判断行为是否发生在有效期的函数
def action_in_valiperd(receive_time, duration, action_time):
    if action_time >= receive_time and action_time <= receive_time+duration*24:
        return 1
    else:
        return 0
    
    
# 创建判断顾客的行为是否收到offer的影响
def offer_has_effect(is_viewed, is_used, has_trans, offer_type):
    if is_viewed == 0: 
        return 0   # 只要用户没看过offer，都算没影响
    else:
        if has_trans == 0:
            return 0   # 如果被看过，但是没有交易，也算没影响 （没有交易，但是用券的情况应该不存在）
        else: 
            if is_used == 1:
                return 1 # 被看过，有交易，且用券，说明有影响
            else: 
                if offer_type == 'informational':
                    return 1 # 被看过，有交易，没用券，但是是消息类的推送（不可能用券），说明有影响
                else:
                    return 0 # 非消息类，即使有交易，没用券也认为没影响

# 根据trans_effect['in_valiperd']和trans_effect['is_effect']两个字段，来判断该交易是否收到offer影响。
def is_trans_effect(in_valiperd, is_effect):
    if pd.isna(is_effect) == False and pd.isna(in_valiperd) == False:
        return in_valiperd*is_effect
    else:
        return 0

    
# 给顾客的income分类
def income_type(x):
    if pd.isna(x):
        return x
    elif x <= 50000:
        return "0~50000"
    elif x > 50000 and x <= 100000:
        return "50001~100000"
    elif x > 100000 and x <= 150000:
        return "100000~150000"
    elif x > 150000 and x <= 200000:
        return "150000~200000"
    else:
        return ">200000"
    
    
# 给顾客的年龄age分类
def age_type(x):
    if pd.isna(x) or x == 118:
        return "None"
    elif x < 10:
        return "0~9y"
    elif x >= 10 and x < 20:
        return "10~19y"
    elif x >= 20 and x < 30:
        return "20~29y"
    elif x >= 30 and x < 40:
        return "30~39y"
    elif x >= 40 and x < 50:
        return "40~49y"
    elif x >= 50 and x < 60:
        return "50~59y"
    elif x >= 60 and x < 70:
        return "60~69y"
    elif x >= 70 and x < 80:
        return "70~79y"
    elif x >= 80 and x < 90:
        return "80~89y"
    elif x >= 90 and x < 100:
        return "90~99y"
    else:
        return ">100y"

### 数据清洗：处理transcript里的value字段，拆分成多个列

In [3]:
# 把value的值拆成多个列（根据字典里的类型）
transcript_value_s = transcript['value'].apply(pd.Series)

# offer id和offer_id这两列重复了，需要把他们合并：如果offer_id为空，则把offer id的值填到offer_id里
transcript_value_s['offer_id']=transcript_value_s.apply(lambda transcript_value_s: combine_offer_id(transcript_value_s['offer id'],transcript_value_s['offer_id']),axis=1)

# 去掉多余的offer id列
transcript_value = transcript_value_s.drop(['offer id'], axis=1)

transcript_value.head()

,amount,offer_id,reward
0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN
1,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN
2,NaN,2906b810c7d4411798c6938adc9daaa5,NaN
3,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN
4,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN


In [4]:
# 把加工好的transcript_value合并到transcript表后面
transcript_new = pd.concat([transcript, transcript_value], axis=1, join_axes=[transcript.index])

# 查看event类型有几种：
# offer received
# offer viewed
# transaction
# offer completed
transcript_new.drop_duplicates(['event'])

,event,person,time,value,amount,offer_id,reward
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN
12650,offer viewed,389bc3fa690240e798340f5a15918d5c,0,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},NaN,f19421c1d4aa40978ebb69ca19b0e20d,NaN
12654,transaction,02c083884c7d45b39cc68e1314fec56c,0,{'amount': 0.8300000000000001},0.83,NaN,NaN
12658,offer completed,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...,NaN,2906b810c7d4411798c6938adc9daaa5,2.0


In [5]:
# 重命名id列，使后面的merge更方便
portfolio.rename(columns={'id':'offer_id'},inplace=True) 
# 重命名id列，使后面的merge更方便
profile.rename(columns={'id':'person_id'},inplace=True) 

# 给顾客的income分类
profile['income_type']=profile.apply(lambda profile: income_type(profile['income']),axis=1)
# 给顾客的年龄age分类
profile['age_type']=profile.apply(lambda profile: age_type(profile['age']),axis=1)

profile.head()

,age,became_member_on,gender,person_id,income,income_type,age_type
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN,NaN,None
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0,100000~150000,50~59y
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN,NaN,None
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0,50001~100000,70~79y
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN,NaN,None


### 1. 计算每个顾客对于每种offer的得分（反应）


In [6]:
# 1.1.1 把收到offer到行为单独提取出来
offer_received = transcript_new[transcript_new['event'] == 'offer received'].loc[:,['person','offer_id','time']]
offer_received.rename(columns={'time':'receive_time','person':'person_id'},inplace=True) 

# 1.1.2 关联portfolio表，取得券的有效期
offer_received = pd.merge(offer_received, portfolio, how='left', on=['offer_id'])


# 1.2.1 把offer view的行为单独提取出来
offer_viewed = transcript_new[transcript_new['event'] == 'offer viewed'].loc[:,['person','offer_id','time']]
offer_viewed.rename(columns={'time':'view_time','person':'person_id'},inplace=True) 

# 1.2.2 收到offer到行为左连接view的行为，来计算多少收到offer后被查看了。（如果只有view行为，没有收到行为，理论上不应该，这里当作脏数据删除了）
offer_received = pd.merge(offer_received, offer_viewed, how='left', on=['person_id','offer_id'])

# 1.2.3 添加一列，判断在有效期内是否有view发生（注意：一条offer的推送可能关联多个view的行为，这里判断每条view行为是否在有效期内）
offer_received['is_viewed']=offer_received.apply(lambda offer_received: 
                                                 action_in_valiperd(offer_received['receive_time'], 
                                                                    offer_received['duration'], 
                                                                    offer_received['view_time']),axis=1)

# 1.2.4 得到每次用户收到offer后，是否在有效期内阅读了。（注意：一条offer用户可能阅读多次，这里统一只计为1-是否阅读；如果后续需要区分阅读次数，可以再改）
offer_received = offer_received.groupby(['person_id','offer_id','receive_time','duration'],as_index=False)['is_viewed'].max()



# 1.3.1 把offer use的行为单独提取出来（跟view的行为处理方法一致）
offer_completed = transcript_new[transcript_new['event'] == 'offer completed'].loc[:,['person','offer_id','time','reward']]
offer_completed.rename(columns={'time':'use_time','person':'person_id'},inplace=True) 

# 1.3.2 收到offer到行为左连接view的行为，来计算多少收到offer后被查看了。（如果只有view行为，没有收到行为，理论上不应该，这里当作脏数据删除了）
offer_received = pd.merge(offer_received, offer_completed, how='left', on=['person_id','offer_id'])

# 1.3.3 添加一列，判断在有效期内是否有use发生（注意：一条offer的推送也可能关联多个use的行为，不一定在同一有效期）
offer_received['is_used']=offer_received.apply(lambda offer_received: 
                                                 action_in_valiperd(offer_received['receive_time'], 
                                                                    offer_received['duration'], 
                                                                    offer_received['use_time']),axis=1)


# 1.3.4 得到每次用户收到offer后，是否在有效期内使用了。（注意：原则上一次推送的offer，只能在有效期内被使用一次，
#      但是也有可能两次推送同样的offer，且有效期重叠，这样就会得到每条推送都关联了2条使用记录，且reward金额可能不同。
#      但是，根据现有数据，没有办法区分哪条use是对应哪条推送的，所以统一都取一条最大的）
offer_received = offer_received.groupby(['person_id','offer_id','receive_time','duration','is_viewed'],as_index=False)['is_used'].max()

#offer_received = offer_received.groupby(['person_id','offer_id','receive_time','duration','is_viewed'],as_index=False)['is_used','reward'].max()
# 注意：使用max有一个问题，就是选取的是每一列的最大值，而不是选取最大值的那一行，也就是说只能在选取单列的最大值的时候才是准确的.
#      所以如果有2条以上使用记录，那么这个reward的金额可能不是真正用那一张的金额


# 1.4.1 把transaction的“交易”行为单独提取出来（这是每个顾客所有的交易记录）
transaction_log = transcript_new[transcript_new['event'] == 'transaction'].loc[:,['person','time','amount']]
transaction_log.rename(columns={'time':'transaction_time','person':'person_id'},inplace=True) 

# 1.4.2 收到offer到行为左连接交易行为，来计算offer的有效期内，是否有交易产生。（这里只看和收到offer的人有关的交易数据）
offer_received = pd.merge(offer_received, transaction_log, how='left', on=['person_id'])

# 1.4.3 添加一列，判断在有效期内是否有use发生（注意：一条offer的推送也可能关联多个use的行为，不一定在同一有效期）
offer_received['has_trans']=offer_received.apply(lambda offer_received: 
                                                 action_in_valiperd(offer_received['receive_time'], 
                                                                    offer_received['duration'], 
                                                                    offer_received['transaction_time']),axis=1)

# 1.4.4 得到每次用户收到offer后，是否在有效期内有交易。（注意：原则上如果有交易，不一定用券；但是用券了，一定有交易）
offer_received = offer_received.groupby(['person_id','offer_id','receive_time','duration','is_viewed','is_used'],as_index=False)['has_trans'].max()
offer_received.head()

,person_id,offer_id,receive_time,duration,is_viewed,is_used,has_trans
0,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,576,7,0,1,1
1,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,336,4,1,0,1
2,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,168,3,1,0,1
3,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408,5,1,1,1
4,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,504,10,1,1,1


In [7]:
# Data-Check：查询是否存在用券了，但是没有交易的情况（理论上不应该）
offer_received.query('is_used == 1 and has_trans == 0')
# 还好没有这种情况，说明数据还是比较完整的。

,person_id,offer_id,receive_time,duration,is_viewed,is_used,has_trans


In [8]:
# 1.5.1 再次关联portfolio表，取得券的类型（后续判断会用到）
offer_received = pd.merge(offer_received, portfolio, how='left', on=['offer_id'])
offer_received = offer_received.drop(['duration_y'], axis=1)
offer_received.rename(columns={'duration_x':'duration'},inplace=True) 

# 1.5.2 添加一列，判断offer对于顾客的影响
offer_received['is_effect']=offer_received.apply(lambda offer_received: 
                                                 offer_has_effect(offer_received['is_viewed'], 
                                                                    offer_received['is_used'], 
                                                                    offer_received['has_trans'],
                                                                    offer_received['offer_type']),axis=1)

# 1.5.3 增加一列is_receive，全部赋值为1，因为所有记录都是收到offer的。这个方便后续的groupby计算
offer_received['is_receive'] = 1
offer_received.head()

,person_id,offer_id,receive_time,duration,is_viewed,is_used,has_trans,channels,difficulty,offer_type,reward,is_effect,is_receive
0,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,576,7,0,1,1,"[web, email, mobile]",10,discount,2,0,1
1,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,336,4,1,0,1,"[web, email, mobile]",0,informational,0,1,1
2,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,168,3,1,0,1,"[email, mobile, social]",0,informational,0,1,1
3,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408,5,1,1,1,"[web, email, mobile, social]",5,bogo,5,1,1
4,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,504,10,1,1,1,"[web, email, mobile, social]",10,discount,2,1,1


In [4]:
#######------------到这里，第一张宽表就做好了。---------------------

### 分析：计算出每个顾客对于每种offer的反应
如果一个顾客，对于offer1，收到3次，有2次有反应，那么该顾客对于次offer的反应score = 2/3

In [9]:
# 1.6.1 计算出每个顾客对于每种offer的反应
user_offer_reaction = offer_received.groupby(['person_id','offer_id'],as_index=False)['is_effect','is_receive'].sum()
user_offer_reaction.head()

,person_id,offer_id,is_effect,is_receive
0,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,0,1
1,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,1,1
2,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,1,1
3,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,1,1
4,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,1,1


In [10]:
user_offer_reaction['effect_score'] = user_offer_reaction['is_effect']/user_offer_reaction['is_receive']
user_offer_reaction.head()

,person_id,offer_id,is_effect,is_receive,effect_score
0,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,0,1,0.0
1,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,1,1,1.0
2,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,1,1,1.0
3,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1.0
4,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,1,1,1.0


## 2. 计算每个顾客的消费分布，是否是offer驱动

In [11]:
# 在1.4.1中，已经把transaction的“交易”行为单独提取出来了（这是每个顾客所有的交易记录）
transaction_log.head()

,person_id,transaction_time,amount
12654,02c083884c7d45b39cc68e1314fec56c,0,0.83
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56
12659,54890f68699049c2a04d415abc25e717,0,13.23
12670,b2f1cd155b864803ad8334cdf13c4bd2,0,19.51
12671,fe97aa22dd3e48c8b143116a8403dd52,0,18.97


In [12]:
# 2.1.1 注意：这里没有交易id，需要把index作为交易的唯一id，因为可能存在同一个人，同一时间，同等金额的交易。
transaction_log['trans_id'] = transaction_log.index
transaction_log.head()

,person_id,transaction_time,amount,trans_id
12654,02c083884c7d45b39cc68e1314fec56c,0,0.83,12654
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,12657
12659,54890f68699049c2a04d415abc25e717,0,13.23,12659
12670,b2f1cd155b864803ad8334cdf13c4bd2,0,19.51,12670
12671,fe97aa22dd3e48c8b143116a8403dd52,0,18.97,12671


In [13]:
transaction_log.shape

(138953, 4)

In [33]:
# 2.2.1 所有交易行为左连接第一张宽表（每个顾客对于每个收到的offer的反应，同一个offer可能收到多次）
trans_effect = pd.merge(transaction_log, offer_received, how='left', on=['person_id'])
trans_effect.head()

,person_id,transaction_time,amount,trans_id,offer_id,receive_time,duration,is_viewed,is_used,has_trans,channels,difficulty,offer_type,reward,is_effect,is_receive
0,02c083884c7d45b39cc68e1314fec56c,0,0.83,12654,0b1e1539f2cc45b7b9fa7c272da2e1d7,408.0,10.0,0.0,0.0,1.0,"[web, email]",20.0,discount,5.0,0.0,1.0
1,02c083884c7d45b39cc68e1314fec56c,0,0.83,12654,ae264e3637204a6fb9bb56bc8210ddfd,0.0,7.0,1.0,0.0,1.0,"[email, mobile, social]",10.0,bogo,10.0,0.0,1.0
2,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,12657,0b1e1539f2cc45b7b9fa7c272da2e1d7,408.0,10.0,1.0,1.0,1.0,"[web, email]",20.0,discount,5.0,1.0,1.0
3,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,12657,2298d6c36e964ae4a3e7e9706d1fb8c2,336.0,7.0,1.0,1.0,1.0,"[web, email, mobile, social]",7.0,discount,3.0,1.0,1.0
4,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,12657,2906b810c7d4411798c6938adc9daaa5,0.0,7.0,1.0,1.0,1.0,"[web, email, mobile]",10.0,discount,2.0,1.0,1.0


In [15]:
# Data-Check：查询是否有没有收到任何offer，但是有交易的顾客
trans_effect[trans_effect.isnull().values==True]

# 有，但是不多。（现实交易里可能会很多）
# 为什么这么多重复行。。。。？？？

,person_id,transaction_time,amount,trans_id,offer_id,receive_time,duration,is_viewed,is_used,has_trans,channels,difficulty,offer_type,reward,is_effect,is_receive
4096,eb540099db834cf59001f83a4561aef3,6,4.74,16379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4096,eb540099db834cf59001f83a4561aef3,6,4.74,16379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4096,eb540099db834cf59001f83a4561aef3,6,4.74,16379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4096,eb540099db834cf59001f83a4561aef3,6,4.74,16379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4096,eb540099db834cf59001f83a4561aef3,6,4.74,16379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4096,eb540099db834cf59001f83a4561aef3,6,4.74,16379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4096,eb540099db834cf59001f83a4561aef3,6,4.74,16379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4096,eb540099db834cf59001f83a4561aef3,6,4.74,16379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4096,eb540099db834cf59001f83a4561aef3,6,4.74,16379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4096,eb540099db834cf59001f83a4561aef3,6,4.74,16379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
trans_effect['is_effect'].dtype

dtype('float64')

In [34]:
# 2.2.2 添加一列，判断该交易是否在offer的有效期内（注意：一条交易可能关联多条offer推送的行为，且可能有多条都在有效期内）
trans_effect['in_valiperd']=trans_effect.apply(lambda trans_effect: 
                                                 action_in_valiperd(trans_effect['receive_time'], 
                                                                    trans_effect['duration'], 
                                                                    trans_effect['transaction_time']),axis=1)

trans_effect.head()

,person_id,transaction_time,amount,trans_id,offer_id,receive_time,duration,is_viewed,is_used,has_trans,channels,difficulty,offer_type,reward,is_effect,is_receive,in_valiperd
0,02c083884c7d45b39cc68e1314fec56c,0,0.83,12654,0b1e1539f2cc45b7b9fa7c272da2e1d7,408.0,10.0,0.0,0.0,1.0,"[web, email]",20.0,discount,5.0,0.0,1.0,0
1,02c083884c7d45b39cc68e1314fec56c,0,0.83,12654,ae264e3637204a6fb9bb56bc8210ddfd,0.0,7.0,1.0,0.0,1.0,"[email, mobile, social]",10.0,bogo,10.0,0.0,1.0,1
2,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,12657,0b1e1539f2cc45b7b9fa7c272da2e1d7,408.0,10.0,1.0,1.0,1.0,"[web, email]",20.0,discount,5.0,1.0,1.0,0
3,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,12657,2298d6c36e964ae4a3e7e9706d1fb8c2,336.0,7.0,1.0,1.0,1.0,"[web, email, mobile, social]",7.0,discount,3.0,1.0,1.0,0
4,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,12657,2906b810c7d4411798c6938adc9daaa5,0.0,7.0,1.0,1.0,1.0,"[web, email, mobile]",10.0,discount,2.0,1.0,1.0,1


In [35]:
# 2.2.3 用in_valiperd和is_effect结合看该笔交易是否收到offer的影响： 只有is_effect和in_valiperd同时为1的情况下，才算。
trans_effect['trans_effect'] = trans_effect.apply(lambda trans_effect: 
                                                 is_trans_effect(trans_effect['in_valiperd'], 
                                                                    trans_effect['is_effect']),axis=1)
trans_effect.head()

,person_id,transaction_time,amount,trans_id,offer_id,receive_time,duration,is_viewed,is_used,has_trans,channels,difficulty,offer_type,reward,is_effect,is_receive,in_valiperd,trans_effect
0,02c083884c7d45b39cc68e1314fec56c,0,0.83,12654,0b1e1539f2cc45b7b9fa7c272da2e1d7,408.0,10.0,0.0,0.0,1.0,"[web, email]",20.0,discount,5.0,0.0,1.0,0,0.0
1,02c083884c7d45b39cc68e1314fec56c,0,0.83,12654,ae264e3637204a6fb9bb56bc8210ddfd,0.0,7.0,1.0,0.0,1.0,"[email, mobile, social]",10.0,bogo,10.0,0.0,1.0,1,0.0
2,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,12657,0b1e1539f2cc45b7b9fa7c272da2e1d7,408.0,10.0,1.0,1.0,1.0,"[web, email]",20.0,discount,5.0,1.0,1.0,0,0.0
3,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,12657,2298d6c36e964ae4a3e7e9706d1fb8c2,336.0,7.0,1.0,1.0,1.0,"[web, email, mobile, social]",7.0,discount,3.0,1.0,1.0,0,0.0
4,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,12657,2906b810c7d4411798c6938adc9daaa5,0.0,7.0,1.0,1.0,1.0,"[web, email, mobile]",10.0,discount,2.0,1.0,1.0,1,1.0


In [36]:
# 2.2.4 得到每笔交易是受到offer影响
trans_effect = trans_effect.groupby(['trans_id','person_id','transaction_time','amount'],as_index=False)['in_valiperd','trans_effect'].max()
trans_effect.head()


,trans_id,person_id,transaction_time,amount,trans_effect
0,12654,02c083884c7d45b39cc68e1314fec56c,0,0.83,0.0
1,12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,1.0
2,12659,54890f68699049c2a04d415abc25e717,0,13.23,0.0
3,12670,b2f1cd155b864803ad8334cdf13c4bd2,0,19.51,1.0
4,12671,fe97aa22dd3e48c8b143116a8403dd52,0,18.97,1.0


In [37]:
trans_effect.shape

(138953, 5)

In [ ]:
###--------到这里，第二张宽表完成了。

##### 分析：结合顾客的属性，分析哪些顾客容易受offer影响

In [38]:
# 2.3.1 结合顾客的属性，分析哪些顾客容易受offer影响
user_trans_effect = pd.merge(trans_effect, profile, how='left', on=['person_id'])
user_trans_effect.head()

,trans_id,person_id,transaction_time,amount,trans_effect,age,became_member_on,gender,income,income_type,age_type
0,12654,02c083884c7d45b39cc68e1314fec56c,0,0.83,0.0,20,20160711,F,30000.0,0~50000,20~29y
1,12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,1.0,42,20160117,M,96000.0,50001~100000,40~49y
2,12659,54890f68699049c2a04d415abc25e717,0,13.23,0.0,36,20171228,M,56000.0,50001~100000,30~39y
3,12670,b2f1cd155b864803ad8334cdf13c4bd2,0,19.51,1.0,55,20171016,F,94000.0,50001~100000,50~59y
4,12671,fe97aa22dd3e48c8b143116a8403dd52,0,18.97,1.0,39,20171217,F,67000.0,50001~100000,30~39y


In [39]:
user_trans_effect.to_excel('user_trans_effect.xlsx')

In [ ]:
# 2.3.2 计算每个顾客的客单价，受影响的订单数，不受影响的订单数，受影响的订单金额，不受影响的订单金额

```python

```